### Sedaro Results API Example

This notebook exercises funcionality of the Sedaro results API tools. The notebook can be pointed at any scenario branch that has previously been simulated. Examples will step through downloading the data and exploring the results.

In [ ]:
import json
from datetime import datetime

from sedaro import SedaroApiClient, SedaroAgentResult, SedaroBlockResult, SedaroSeries

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [ ]:
# Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('./config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
SCENARIO_BRANCH_ID = config['SCENARIO_BRANCH_ID']  # ID of the scenario template branch
HOST = config['HOST']                              # Sedaro instance URL

### Downloading and Navigating Results

The primary entrypoint of the results API is accessed via the `simulation` property on a `ScenarioBranch` retrieved using the `SedaroApiClient`. 

In [ ]:
sedaro = SedaroApiClient(API_KEY, HOST)

sim = sedaro.scenario(SCENARIO_BRANCH_ID).simulation

This class offers a few methods for pulling data from scenarios. The most commonly-used method is `.results` that pulls the latest results into a new result object. If the simulation is not complete, the resulting object will indicate the status is "Running" and not contain any results. Alternatively, use the `.results_poll` method to wait for an in-progress simulation to complete and download results after.

Any object in the results API will provide a descriptive summary of its contents when the `.summarize` method is called.

In [ ]:
# Query scenario and do not wait if still running
result = sim.results()

# Query scenario and wait for it to finish
# result = sim.results_poll()

result.summarize()

Top-level results are broken down by agent. You can view results for any agent (templated or peripheral) by calling the `.agent` method with the name of an agent listed above. The result class also has helper methods containing lists of the available agents: `.templated_agents` and `peripheral_agents`.

In [ ]:
agent_result = result.agent(result.templated_agents[0])
agent_result.summarize()

Agent results are organized by blocks, where blocks are the basic elements that make up each simulation. Results for any blocks can be queried by a complete or partial ID, as listed in the ID column above. Note that partial IDs will only return a result if it matches the beginning of only one block ID. The block result class also has a helper attribute, `.blocks`, that returns a list of all block IDs contained in that instance.

There is one special entry in the block list: `root`. This index captures all variables that exist on the lowest level of the simulated agent and do not belong to any block. For example, the agent `position` will usually exist in `root`.

In [ ]:
block_result = agent_result.block('root')
block_result.summarize()

Block results contain the time histories of state variables associated with that block. State variables can be queried directly through the `.<VARIABLE_NAME>` attribute or the `.variable` method. The `.variables` attribute returns a list of the names of all variables available in the object.

Querying a particular variable returns a time series object.

In [ ]:
series = block_result.perturbationTorque
series.summarize()

Series allow direct access to the data through three attributes:
* `.mjd`: Returns a list of epochs as Modified Julian Dates.
* `.elapsed_time`: Returns a list of elapsed times from the initial epoch in seconds.
* `.values`: Returns a list of state variable values corresponding to the times above.

Alternatively, a series is iterable and can be traversed as shown below.

In [ ]:
for mjd, elapsed_time, value in series:
    print(mjd, elapsed_time, value)
    if elapsed_time > 10:
        break

Sometimes, a series contains multiple sub-series. This can happen when, for example, multiple frame or unit representations are available for that series. A subseries can be queried through the `.<SERIES_NAME>` attribute and will return a new series.

In [ ]:
block_result.position.summarize()

In [ ]:
block_result.position.ecef.summarize()

### Saving and Loading Results Locally

Every Sedaro result object supports saving and restoring results to and from a file. This functionality can be helpful when working with large results to avoid re-downloading the same data. To save data, call the `.to_file` method on any result object. Note that the save operation will return an error if the file already exists.

In [ ]:
filename = f'SimulationResult_{datetime.now().strftime("%d%m%Y_%H%M%S")}.bak'
result.to_file(filename)

Use the `.from_file` class method to load previously-saved results.

In [ ]:
new_result = SedaroSimulationResult.from_file(filename)
new_result.summarize()

Saving and loading also works for agent, block, and series results using the same methods. Uncomment examples below to try the functionality for different types.

In [ ]:
# Saving/loading reference examples

# agent_result.to_file('AgentResult.bak')
# new_agent_result = SedaroAgentResult.from_file('AgentResult.bak')

# block_result.to_file('BlockResult.bak')
# new_block_result = SedaroBlockResult.from_file('BlockResult.bak')

# series.to_file('SeriesResult.bak')
# new_series = SedaroSeries.from_file('SeriesResult.bak')

### Plotting

Series objects also offer some basic plotting functionality for quick visualizations of data through integration with `matplotlib`. If `matplotlib` is not available, the commands below will raise an exception. In some cases, the underlying data is not a plottable data type and the commands below will raise an exception indicating that the operation could not complete.

The `.plot` command accepts arbitrary formatting keyword arguments supported by `matplotlib.pyplot.plot` and will pass them directly to that function.

In [ ]:
block_result.position.eci.plot(linewidth=1)

An optional keyword argument, `show`, allows for plotting multiple series on the same figure. By default, the `.plot` command will show the figure immediately after plotting. With `show=False`, the plot command will not display the figure until a future command uses `show=True` or the user explicitly calls `matplotlib.pyplot.show()`.

In [ ]:
block_result.position.eci.plot(linewidth=1, show=False)
block_result.position.ecef.plot(linewidth=1)